In [1]:
import pandas as pd
import signal
from mip import Model, xsum, maximize, BINARY
from itertools import product
import time
import numpy as np

In [2]:
# day start at 8 and ends at 14
data ={"Time":[60,40,80,50,60,30, 60, 120, 30, 60, 120],
"Activity name":["watch anime","exercise","go to swim", "go running","have lunch","have breakfast",
                 "play video game short","play video game long","read a good book",
                 "do a coding tutorial short", "do a coding tutorial long"],

"Satisfaction":[7,7,8,8,3,3,7,8,6,9,10],
"Energy needed":[0,3,4,5,0,0,0,0,3,6,8]
}
df = pd.DataFrame.from_dict(data)

# Specify min and max start time (hour)
df.loc[df["Activity name"].str.contains("breakfast"),"End Before"] = 11
df.loc[df["Activity name"].str.contains("lunch"),"Start After"] = 12
df.loc[df["Activity name"].str.contains("exercise|swim|run|coding"),"Start After"] = 9

# Specify if an activity must be done in the time window
df["Must_Do"] = False
df.loc[df["Activity name"].str.contains("breakfast|lunch"),"Must_Do"] = True

In [3]:
df

,Time,Activity name,Satisfaction,Energy needed,End Before,Start After,Must_Do
0,60,watch anime,7,0,NaN,NaN,False
1,40,exercise,7,3,NaN,9.0,False
2,80,go to swim,8,4,NaN,9.0,False
3,50,go running,8,5,NaN,9.0,False
4,60,have lunch,3,0,NaN,12.0,True
5,30,have breakfast,3,0,11.0,NaN,True
6,60,play video game short,7,0,NaN,NaN,False
7,120,play video game long,8,0,NaN,NaN,False
8,30,read a good book,6,3,NaN,NaN,False
9,60,do a coding tutorial short,9,6,NaN,9.0,False


In [4]:
# df.to_csv("morning_activies.csv")

In [82]:
df

,Time,Activity name,Satisfaction,Energy needed,End Before,Start After,Must_Do,Starting_Time_min,Starting_Time,Ending_Time
0,60,watch anime,7,0,NaN,NaN,False,661.0,11h 1m,12h 1m
1,40,exercise,7,3,NaN,9.0,False,771.0,12h 51m,13h 31m
2,80,go to swim,8,4,NaN,9.0,False,675.0,11h 15m,12h 35m
3,50,go running,8,5,NaN,9.0,False,651.0,10h 51m,11h 41m
4,60,have lunch,3,0,NaN,12.0,True,769.0,12h 49m,13h 49m
5,30,have breakfast,3,0,11.0,NaN,True,623.0,10h 23m,10h 53m
6,60,play video game short,7,0,NaN,NaN,False,665.0,11h 5m,12h 5m
7,120,play video game long,8,0,NaN,NaN,False,714.0,11h 54m,13h 54m
8,30,read a good book,6,3,NaN,NaN,False,NaN,NaN,NaN
9,60,do a coding tutorial short,9,6,NaN,9.0,False,NaN,NaN,NaN


In [34]:
df.loc[2,"Start After"]

9.0

In [52]:
# df = pd.read_csv("morning_activies.csv")

# starting
start = time.time()

# Metadata
day_start = 8 # day starts at 8 Am

##########################
#  SETS
##########################

# Set of activities
I = set(range(df.shape[0]))

# Set of time instant in minutes, time start at 8AM and ends at 14
T = set(range(360))

# sets of activities i that has a start date, a end date, must be done
S = set(df[df["Start After"].notnull()].index)
E = set(df[df["End Before"].notnull()].index)

##########################
#  Parameters
##########################

# Satisfaction of each activity
w = df["Satisfaction"].tolist()

# Energy cost
e = df["Energy needed"].tolist()

# Time needed to complete the activity
d = df["Time"]

# Max energy, since I am not a morning howl i specificy a max energy available ;)
Me = 12

# activies must be done 
O = df["Must_Do"]*1

##########################
#  Model definition
##########################

m = Model("morning_schedule")

# binary variables indicating if activity i starts at time t
x = [[m.add_var(name='activity_start_in_t',var_type=BINARY) for t in T] for i in I]

# binary variables indicating if activity i is done
y = [m.add_var(name='activity_done') for i in I]


m.objective = maximize(xsum(y[i]*w[i] for i in I))

# minimum number of people in office satisfied

for i in I: # compulsory activity should be done, y>=0
    m += y[i] >= O[i]

for i in I: # y <= 1
    m += y[i] <= 1
# max energy
m += xsum(y[i]*e[i] for i in I) <= Me

for i in S : # min start
    m += xsum(t*x[i][t] for t in T) >= (df.loc[i,"Start After"]-day_start)*60*y[i]

for i in E : # max end
    m += xsum(t*x[i][t] for t in T) <= ((df.loc[i,"End Before"]-day_start)*60 - d[i]) + len(T)*(1-y[i])

for i in I: # Link y and x variables, y=1 => exist and x=1
    m += y[i] == xsum(x[i][t]  for t in T)

for i in I:  # do one activity at most once
    m += xsum(x[i][t]  for t in T) <= 1

    
for (i, t) in product(I,T): # do at most one activity at once part 1
    m += x[i][t] <= 1 - xsum(x[j][tau] if (j!=i & tau>=t-d[j]+1) else 0
                             for j in I for tau in set(range(t+d[i]))) #{t-d[j]+1, ... t+d[i]-1}


m.optimize()

intermediate = time.time()
print(f"intermediate time elapsed: {intermediate - start}")

df["Starting_Time_min"]=np.nan
#df["Ending_Time"]=np.nan

change_i = ""
for i,t in product(I,T):
    if(i!=change_i):
        if(y[i].x==0):
            change_i = i
            continue  
        if(x[i][t].x>0):
           df.loc[i,"Starting_Time_min"] = 60*8 + t
           #df.loc[i,"Ending_Time"] = 60*8 + t + d[i]
           change_i = i
           print(f"i: {i}, t: {t}")

end = time.time()
print(f"time elapsed: {end - start}")
print(f"optimal solution provide satisfaction: {m.objective_value}")

. -52 iterations 53
Cbc0038I Pass  33: suminf.    0.49776 (2) obj. -51.2399 iterations 30
Cbc0038I Pass  34: suminf.    0.66667 (2) obj. -52 iterations 1
Cbc0038I Pass  35: suminf.    2.60000 (7) obj. -52.2 iterations 88
Cbc0038I Pass  36: suminf.    0.49776 (2) obj. -51.2399 iterations 64
Cbc0038I Pass  37: suminf.    0.49776 (2) obj. -51.2399 iterations 3
Cbc0038I Pass  38: suminf.    0.66667 (2) obj. -52 iterations 12
Cbc0038I Pass  39: suminf.    0.66667 (2) obj. -52 iterations 92
Cbc0038I Pass  40: suminf.    0.49776 (2) obj. -51.2399 iterations 54
Cbc0038I Pass  41: suminf.    0.66667 (2) obj. -52 iterations 8
Cbc0038I Pass  42: suminf.    2.66667 (8) obj. -52 iterations 75
Cbc0038I Pass  43: suminf.    0.49776 (2) obj. -51.2399 iterations 64
Cbc0038I Pass  44: suminf.    0.66667 (2) obj. -52 iterations 15
Cbc0038I Pass  45: suminf.    0.66667 (2) obj. -52 iterations 52
Cbc0038I Pass  46: suminf.    0.49776 (2) obj. -51.2399 iterations 40
Cbc0038I Pass  47: suminf.    0.66667 (2)

Cbc0038I Pass  32: suminf.    0.66667 (2) obintermediate time elapsed: 47.364431381225586
i: 0, t: 280
i: 1, t: 291
i: 2, t: 342
i: 3, t: 293
i: 4, t: 289
i: 5, t: 104
i: 6, t: 182
i: 7, t: 307
time elapsed: 47.36721110343933


In [73]:
df["Starting_Time"] = df["Starting_Time_min"]
df["Ending_Time"] = df["Starting_Time_min"] + df["Time"]


df.loc[~df["Starting_Time"].isnull(),"Starting_Time"] = (
    (df.loc[~df["Starting_Time"].isnull(),"Starting_Time"]//60).astype(int).astype(str)
    + "h " 
    + (df.loc[~df["Starting_Time"].isnull(),"Starting_Time"]%60).astype(int).astype(str) + "m")

df.loc[~df["Ending_Time"].isnull(),"Ending_Time"] = (
    (df.loc[~df["Ending_Time"].isnull(),"Ending_Time"]//60).astype(int).astype(str)
    + "h " 
    + (df.loc[~df["Ending_Time"].isnull(),"Ending_Time"]%60).astype(int).astype(str) + "m")

In [74]:
df.sort_values(by="Starting_Time_min")[
    ["Starting_Time_min","Activity name","Starting_Time",
     "Ending_Time", "Energy needed","Satisfaction"]]

,Starting_Time_min,Activity name,Starting_Time,Ending_Time,Energy needed,Satisfaction
5,623.0,have breakfast,10h 23m,10h 53m,0,3
3,651.0,go running,10h 51m,11h 41m,5,8
0,661.0,watch anime,11h 1m,12h 1m,0,7
6,665.0,play video game short,11h 5m,12h 5m,0,7
2,675.0,go to swim,11h 15m,12h 35m,4,8
7,714.0,play video game long,11h 54m,13h 54m,0,8
4,769.0,have lunch,12h 49m,13h 49m,0,3
1,771.0,exercise,12h 51m,13h 31m,3,7
8,NaN,read a good book,NaN,NaN,3,6
9,NaN,do a coding tutorial short,NaN,NaN,6,9


In [75]:
df.loc[df["Starting_Time"].notna(), "Satisfaction"].sum()

51

In [76]:
for i in range(11):
    if(y[i].x==1):
        print(f"y[{i}] = {y[i].x}")

y[0] = 1.0
y[1] = 1.0
y[2] = 1.0
y[3] = 1.0
y[4] = 1.0
y[5] = 1.0
y[6] = 1.0
y[7] = 1.0


In [77]:
skip_i = ""
for i,t in product(I,T):
    if(skip_i!=i):
        if(y[i].x!=1):
            skip_i = i
            continue  
        if(x[i][t].x>0):
           change_i = i
           print(f"x[{i},{t+8*60}] = {x[i][t].x}")
            

x[0,661] = 1.0
x[1,771] = 1.0
x[2,675] = 1.0
x[3,651] = 1.0
x[4,769] = 1.0
x[5,623] = 1.0
x[6,665] = 1.0
x[7,714] = 1.0
